In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pandas import DataFrame, read_csv
import json

import sys
sys.path.insert(0, '../ToolBox')

import CrossValidation, MLP, Helper, ZScore, RandomForest

import os, sys
import cPickle

In [ ]:
saved_train_data = open('pre_processed_data/train_preprocessed.pkl', 'rb')
normalized_train_data = cPickle.load(saved_train_data)

In [ ]:
saved_test_data = open('pre_processed_data/test_preprocessed.pkl', 'rb')
normalized_test_data = cPickle.load(saved_test_data)

In [ ]:
import yaml

with open("../ToolBox/params.json") as data_file:
    params = yaml.safe_load(data_file)
print 'json loaded'
labels = normalized_train_data[:,0]
print labels.shape
labels = np.reshape(labels, [labels.shape[0], 1])
print labels.shape
labels = np.subtract(np.array(labels), 1)
data_dict = dict({'x':normalized_train_data[:,1:], 'y':Helper.convert_one_hot_encoding(labels,params['MLPandRF']['MLP']['output-dim'])})

print data_dict['y'].shape

mlp = MLP.init(params['MLPandRF']['MLP'])
mlp['train'](data_dict)

data_dict['x'] = mlp['predict'](data_dict['x'], 1)
data_dict['y'] = labels

test_transformed = mlp['predict'](normalized_test_data, 1)

rf = RandomForest.init(params['MLPandRF']['RF'])
model = rf['train'](data_dict)
result_dict = rf['predict_probabilities'](test_transformed, model)
result_dict

In [ ]:
#lis = []
#lis.append("ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer")

#for idx, z in enumerate(result_dict["network_0"]):
#    zList = z[0].tolist()
#    temp_str = str(idx+1)
#    for idy, x in enumerate(zList):
#        temp_str = temp_str+", "+str(x)
#    lis.append(temp_str)


#lis = np.array(lis)
#np.savetxt('result_shelter_animal_outcomes.csv', lis, fmt='%s', delimiter=',')